In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import pprint
import os
import clip
import imagehash
import cv2
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import pickle
import shutil
import torch
import re
from string import punctuation
import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("stopwords")
from torch.utils.data import Dataset, DataLoader

stopwords_ = set(stopwords.words("english"))



[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("../data/train_data_v2.csv")

In [3]:
df.head()

,caption,product,image_file_path,image_parent_folder,img_ext,full_path,single_pair_cosine
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.425996
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.405180
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.409037
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.450119
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.460671


In [4]:
df["step_1_lower"] = df["caption"].apply(lambda x:x.lower())
df.head()

,caption,product,image_file_path,image_parent_folder,img_ext,full_path,single_pair_cosine,step_1_lower
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.425996,z-couple shook his head ornaments doll car dec...
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.405180,z remote anti-theft door wireless door anti-th...
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.409037,z qinghai qing liang xidan tip outside microme...
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.450119,z magic incognito clothes stack pants multi-la...
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.460671,z ge zi yong pin yong ju ge zi zhuang men tie ...


In [5]:
df["step_2_no_sp_char"] = df["step_1_lower"].apply(lambda x:re.sub(r"[^A-Za-z0-9\s]+", "", x))
df.head()

,caption,product,image_file_path,image_parent_folder,img_ext,full_path,single_pair_cosine,step_1_lower,step_2_no_sp_char
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.425996,z-couple shook his head ornaments doll car dec...,zcouple shook his head ornaments doll car deco...
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.405180,z remote anti-theft door wireless door anti-th...,z remote antitheft door wireless door antithef...
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.409037,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.450119,z magic incognito clothes stack pants multi-la...,z magic incognito clothes stack pants multilay...
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.460671,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...


In [6]:
def no_stop(x):
    tokens = word_tokenize(x)
    clean_tokens = [t for t in tokens if not t in stopwords_]
    clean_text = " ".join(clean_tokens)
    return clean_text
df["step_3_no_stop"] = df["step_2_no_sp_char"].apply(no_stop)
df.head()

,caption,product,image_file_path,image_parent_folder,img_ext,full_path,single_pair_cosine,step_1_lower,step_2_no_sp_char,step_3_no_stop
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.425996,z-couple shook his head ornaments doll car dec...,zcouple shook his head ornaments doll car deco...,zcouple shook head ornaments doll car decorati...
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.405180,z remote anti-theft door wireless door anti-th...,z remote antitheft door wireless door antithef...,z remote antitheft door wireless door antithef...
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.409037,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.450119,z magic incognito clothes stack pants multi-la...,z magic incognito clothes stack pants multilay...,z magic incognito clothes stack pants multilay...
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.460671,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...


In [7]:
df["step_4_no_num"] = df["step_3_no_stop"].apply(lambda x: re.sub(r"\b[0-9]+\b\s*", "", x))
df.head()

,caption,product,image_file_path,image_parent_folder,img_ext,full_path,single_pair_cosine,step_1_lower,step_2_no_sp_char,step_3_no_stop,step_4_no_num
0,Z-Couple Shook His Head Ornaments Doll Car Dec...,O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.425996,z-couple shook his head ornaments doll car dec...,zcouple shook his head ornaments doll car deco...,zcouple shook head ornaments doll car decorati...,zcouple shook head ornaments doll car decorati...
1,Z Remote Anti-Theft Door Wireless Door Anti-Th...,O1CN01eFgq841CxMtYCpqWF_!!147-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.405180,z remote anti-theft door wireless door anti-th...,z remote antitheft door wireless door antithef...,z remote antitheft door wireless door antithef...,z remote antitheft door wireless door antithef...
2,Z Qinghai qing liang Xidan Tip outside Microme...,O1CN01fXCPeA1SzgmI1mVLf_!!2207662892318.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.409037,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...,z qinghai qing liang xidan tip outside microme...
3,Z Magic Incognito Clothes Stack Pants Multi-La...,O1CN01qgixtq1BtPqrInOuX_!!3-0-lubanu.jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,jpg,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.450119,z magic incognito clothes stack pants multi-la...,z magic incognito clothes stack pants multilay...,z magic incognito clothes stack pants multilay...,z magic incognito clothes stack pants multilay...
4,z ge zi yong pin yong ju ge zi zhuang men tie ...,TB1ZqfODEY1gK0jSZFCq6AwqXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,train_text_img_pairs_0/train_text_img_pairs_0_...,qXXa,/home/ubuntu/Desktop/CVPR 2022 AliProducts Cha...,0.460671,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...,z ge zi yong pin yong ju ge zi zhuang men tie ...


In [7]:
df.shape

(4099784, 6)

In [12]:
df = pd.read_csv("./train_data_v1.csv")

In [14]:
df["reletive_path"][0]

'train_text_img_pairs_0/train_text_img_pairs_0_compressed/O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg'

In [8]:
df["reletive_path"] = df["image_parent_folder"] + df["product"]


In [10]:
df["reletive_path"][0]

'train_text_img_pairs_0/train_text_img_pairs_0_compressed/O1CN01ogOt7d1DHyjGtaaWT_!!192-0-lubanu.jpg'

In [11]:
df.to_csv("./train_data_v1.csv")

In [ ]:
# df["step_5_no_dupe"] = df["step_4_no_num"].apply(lambda x:" ".join(set(word_tokenize(x))))
# df.head()

In [ ]:
# def no_short_token(x):
#     tokens = word_tokenize(x)
#     clean_tokens = [t for t in tokens if len(t) > 1]
#     clean_text = " ".join(clean_tokens)
#     return clean_text
# df["step_6_no_len_1_token"] = df["step_5_no_dupe"].apply(lambda x:" ".join(set(word_tokenize(x))))
# df.head()

In [ ]:
# def crop_token(x):
#     tokens = x.split(" ")
#     total_len = len(tokens)
#     if total_len >3:
#         percent_70 = round(total_len*0.7)
#         crop_text = tokens[0:percent_70]
#         return " ".join(crop_text)
#     else:
#         return " ".join(tokens)
# df["caption_crop_first_70%"] = df["caption"].apply(crop_token)
# df.head()

In [ ]:
# def crop_token(x):
#     tokens = x.split(" ")
#     total_len = len(tokens)
#     if total_len > 3:
#         percent_70 = round(total_len*0.7)
#         crop_text = tokens[-percent_70:]
#         return " ".join(crop_text)
#     else:
#         return " ".join(tokens)
# df["caption_crop_last_70%"] = df["caption"].apply(crop_token)
# df.head()

In [ ]:
# def crop_token(x):
#     tokens = x.split(" ")
#     total_len = len(tokens)
#     if total_len >3:
#         percent_70 = round(total_len*0.7)
#         strt_idx = (total_len // 2) - (percent_70 // 2)
#         end_idx = (total_len // 2) + (percent_70 // 2)
#         crop_text = tokens[strt_idx:end_idx+1]
#         return " ".join(crop_text)
#     else:
#         return " ".join(tokens)
# df["caption_crop_middle_70%"] = df["caption"].apply(crop_token)
# df.head()

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from tqdm import tqdm
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh")

# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")
# model.to("cuda")

In [ ]:
# caption_loader  = DataLoader(df["caption"].values,batch_size=64,shuffle=False,num_workers=12)

In [ ]:
# def translate_to_cn(txt):
#     with torch.no_grad():
#         token = tokenizer.prepare_seq2seq_batch(txt,return_tensors='pt')
#         token = token.to("cuda")
#         pred = model.generate(**token)
#         pred = pred.detach().cpu()
#         return pred


# def translate_to_cn_no_decode(txt):
#     with torch.no_grad():
#         token = tokenizer.prepare_seq2seq_batch(txt,return_tensors='pt')
#         token = token.to("cuda")
#         pred = model.generate(**token)
#         pred = pred.detach().cpu()
#         return pred

# df["caption_cn"] = df["caption"].apply(translate_to_cn)
# df.head()

In [ ]:
# all_transalated_txt = []
# for b in tqdm(caption_loader):
#     transalated_txt = translate_to_cn(b)
#     transalated_txt = tokenizer.batch_decode(transalated_txt)
#     transalated_txt = [t.replace("<pad>","") for t in transalated_txt]
#     transalated_txt = np.array(transalated_txt)
#     all_transalated_txt.append(transalated_txt)

In [ ]:
# all_transalated_txt_concat = np.concatenate(all_transalated_txt)

In [ ]:
# all_transalated_txt_concat.shape

In [ ]:
# filename = "./translated_caption.pkl"
# file = open(filename,"wb")
# pickle.dump(all_transalated_txt,file)
# file.close()

In [ ]:
# df["captions_cn"] = all_transalated_txt_concat

In [ ]:
# df.head()

In [9]:
df.shape

(3930717, 11)

In [10]:
df.to_csv("../data/train_data_v3.csv",index=False)